In [1]:
import glob
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
from src.func_data_engineering import *

# auto load modules
%load_ext autoreload
%autoreload

%cd C:\Users\tyler\repos\airbnb-forecast

C:\Users\tyler\repos\airbnb-forecast


In [2]:
# store raw data
# listing

cols = ['id', 'last_scraped', 'host_id', 'host_since','host_neighbourhood',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'calculated_host_listings_count',
     'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms',
 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 
         'street',
      'neighbourhood_cleansed','city', 'state', 'zipcode', 'market',
       'smart_location','country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'number_of_reviews', 'number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value','instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 
        'reviews_per_month']

path = r'data\20'
all_files = glob.glob(os.path.join(path,"listings*.csv"))

ls_file = []
for file in all_files:
    df = pd.read_csv(file,header=0)
    df = df[cols]
    ls_file.append(df)

df_list = pd.concat(ls_file,axis=0,ignore_index=True)

C:\Users\tyler\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\tyler\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [160]:
engine = connect_my_db('secrets/db_string')

In [166]:
fs_upload(engine,df_ls,'FS_LIST')

85.58639979362488

In [167]:
fs_upload(engine,df_host,'FS_HOST')

73.12233901023865

In [168]:
fs_upload(engine,df_review,'FS_REVIEW')

106.92497611045837

In [169]:
fs_upload(engine,df_location,'FS_LOCATION_RAW')

81.0525414943695

In [ ]:
# data cleaning

# column names
df_list.columns = df_list.columns.str.upper()

colname_map = {'CALCULATED_HOST_LISTINGS_COUNT':'HOST_LISTINGS_COUNT',
       'CALCULATED_HOST_LISTINGS_COUNT_ENTIRE_HOMES':'HOST_ENTIRE_HOMES',
       'CALCULATED_HOST_LISTINGS_COUNT_PRIVATE_ROOMS':'HOST_PRIVATE_ROOMS',
       'CALCULATED_HOST_LISTINGS_COUNT_SHARED_ROOMS':'HOST_SHARED_ROOMS',
              'LAST_SCRAPED':'SCRAPED_DATE'}
df_list = df_list.rename(columns=colname_map)

# str columns lowercase
df_list.PROPERTY_TYPE=df_list.PROPERTY_TYPE.str.lower()
df_list.ROOM_TYPE =df_list.ROOM_TYPE.str.lower()
df_list.BED_TYPE =df_list.BED_TYPE.str.lower()
df_list.CANCELLATION_POLICY =df_list.CANCELLATION_POLICY.str.lower()

# convert 't'and 'f' to 1 and 0
df_list.INSTANT_BOOKABLE = df_list.INSTANT_BOOKABLE.replace({'t':1,'f':0})
df_list.REQUIRE_GUEST_PROFILE_PICTURE = df_list.REQUIRE_GUEST_PROFILE_PICTURE.replace({'t':1,'f':0})
df_list.REQUIRE_GUEST_PHONE_VERIFICATION = df_list.REQUIRE_GUEST_PHONE_VERIFICATION.replace({'t':1,'f':0})
df_list.HOST_IS_SUPERHOST = df_list.HOST_IS_SUPERHOST.replace({'t':1,'f':0})
df_list.HOST_HAS_PROFILE_PIC = df_list.HOST_HAS_PROFILE_PIC.replace({'t':1,'f':0})
df_list.HOST_IDENTITY_VERIFIED = df_list.HOST_IDENTITY_VERIFIED.replace({'t':1,'f':0})
df_list.IS_LOCATION_EXACT = df_list.IS_LOCATION_EXACT.replace({'t':1,'f':0})

# convert str to num
df_list.HOST_RESPONSE_RATE = df_list.HOST_RESPONSE_RATE.str.replace('%','').astype(float)
df_list.HOST_ACCEPTANCE_RATE = df_list.HOST_ACCEPTANCE_RATE.str.replace('%','').astype(float)




# feature store

#todo: experiment on keep more values
df_list.PROPERTY_TYPE=df_list.PROPERTY_TYPE.replace(['loft','guest suite','condominium'],'apartment')
df_list.PROPERTY_TYPE=df_list.PROPERTY_TYPE.replace(
    ['townhouse','guesthouse','cottage','bungalow','cabin','tiny house','nature lodge','villa','earth house','castle', 
     'treehouse', 'chalet','lighthouse'],'house')
df_list.PROPERTY_TYPE=df_list.PROPERTY_TYPE.replace(
    ['farm stay','bed and breakfast','barn','train', 'boat', 'camper/rv','tent','hut','kezhan (china)',
     'casa particular (cuba)','campsite', 'dome house', 'tipi'],'other')
df_list.PROPERTY_TYPE=df_list.PROPERTY_TYPE.replace(
    ['boutique hotel','serviced apartment','hotel','hostel','aparthotel'],'serviced')

# create luxury_flag
df_list['LUXURY_FLAG']= np.where(df_list.CANCELLATION_POLICY.str.contains('luxury'),1,0)


In [159]:
# feature store

# FS_LISTING
df_ls = df_list[['ID','HOST_ID','LAST_SCRAPED',
                 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES',
       'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'AMENITIES', 'SQUARE_FEET',
       'INSTANT_BOOKABLE', 'IS_BUSINESS_TRAVEL_READY','GUESTS_INCLUDED',
       'CANCELLATION_POLICY', 'REQUIRE_GUEST_PROFILE_PICTURE',
       'REQUIRE_GUEST_PHONE_VERIFICATION',
       'LUXURY_FLAG','NEIGHBOURHOOD_CLEANSED','LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT',]]

df_ls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 23 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID                                148835 non-null  int64  
 1   HOST_ID                           148835 non-null  int64  
 2   LAST_SCRAPED                      148835 non-null  object 
 3   PROPERTY_TYPE                     148835 non-null  object 
 4   ROOM_TYPE                         148835 non-null  object 
 5   ACCOMMODATES                      148835 non-null  int64  
 6   BATHROOMS                         148718 non-null  float64
 7   BEDROOMS                          148606 non-null  float64
 8   BEDS                              147612 non-null  float64
 9   BED_TYPE                          148835 non-null  object 
 10  AMENITIES                         148835 non-null  object 
 11  SQUARE_FEET                       1044 non-null    f

In [128]:
# FS_HOST


df_host = df_list[['ID', 'LAST_SCRAPED', 'HOST_ID', 'HOST_SINCE', 'HOST_NEIGHBOURHOOD',
       'HOST_RESPONSE_TIME', 'HOST_RESPONSE_RATE', 'HOST_ACCEPTANCE_RATE',
       'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT',
       'HOST_ENTIRE_HOMES',
       'HOST_PRIVATE_ROOMS',
       'HOST_SHARED_ROOMS', 'HOST_VERIFICATIONS',
       'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED']]

df_host.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      148835 non-null  int64  
 1   LAST_SCRAPED            148835 non-null  object 
 2   HOST_ID                 148835 non-null  int64  
 3   HOST_SINCE              148751 non-null  object 
 4   HOST_NEIGHBOURHOOD      94904 non-null   object 
 5   HOST_RESPONSE_TIME      99604 non-null   object 
 6   HOST_RESPONSE_RATE      99604 non-null   float64
 7   HOST_ACCEPTANCE_RATE    93013 non-null   float64
 8   HOST_IS_SUPERHOST       148751 non-null  float64
 9   HOST_LISTINGS_COUNT     148835 non-null  int64  
 10  HOST_ENTIRE_HOMES       148835 non-null  int64  
 11  HOST_PRIVATE_ROOMS      148835 non-null  int64  
 12  HOST_SHARED_ROOMS       148835 non-null  int64  
 13  HOST_VERIFICATIONS      148835 non-null  object 
 14  HOST_HAS_PROFILE_PIC

In [144]:
# fs reviews

df_review = df_list[['ID', 'LAST_SCRAPED','NUMBER_OF_REVIEWS',
                     'FIRST_REVIEW',
       'LAST_REVIEW', 'REVIEW_SCORES_RATING', 'REVIEW_SCORES_ACCURACY',
       'REVIEW_SCORES_CLEANLINESS', 'REVIEW_SCORES_CHECKIN',
       'REVIEW_SCORES_COMMUNICATION', 'REVIEW_SCORES_LOCATION',
       'REVIEW_SCORES_VALUE','REVIEWS_PER_MONTH']]

df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           148835 non-null  int64  
 1   LAST_SCRAPED                 148835 non-null  object 
 2   NUMBER_OF_REVIEWS            148835 non-null  int64  
 3   FIRST_REVIEW                 118283 non-null  object 
 4   LAST_REVIEW                  118283 non-null  object 
 5   REVIEW_SCORES_RATING         115283 non-null  float64
 6   REVIEW_SCORES_ACCURACY       115171 non-null  float64
 7   REVIEW_SCORES_CLEANLINESS    115204 non-null  float64
 8   REVIEW_SCORES_CHECKIN        115123 non-null  float64
 9   REVIEW_SCORES_COMMUNICATION  115189 non-null  float64
 10  REVIEW_SCORES_LOCATION       115129 non-null  float64
 11  REVIEW_SCORES_VALUE          115119 non-null  float64
 12  REVIEWS_PER_MONTH            118283 non-null  float64
dtyp

In [170]:
# fs location

df_location = df_list[['ID', 'SCRAPED_DATE','STREET','NEIGHBOURHOOD_CLEANSED','CITY', 'STATE', 'ZIPCODE', 'MARKET',
       'SMART_LOCATION', 'COUNTRY', 'LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT']]

KeyError: "['SCRAPED_DATE'] not in index"

In [165]:
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   STREET                  148835 non-null  object 
 1   NEIGHBOURHOOD_CLEANSED  148835 non-null  object 
 2   CITY                    148739 non-null  object 
 3   STATE                   148261 non-null  object 
 4   ZIPCODE                 148238 non-null  object 
 5   MARKET                  148669 non-null  object 
 6   SMART_LOCATION          148835 non-null  object 
 7   COUNTRY                 148835 non-null  object 
 8   LATITUDE                148835 non-null  float64
 9   LONGITUDE               148835 non-null  float64
 10  IS_LOCATION_EXACT       148835 non-null  int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 12.5+ MB


In [129]:
df_list.columns

Index(['ID', 'LAST_SCRAPED', 'HOST_ID', 'HOST_SINCE', 'HOST_NEIGHBOURHOOD',
       'HOST_RESPONSE_TIME', 'HOST_RESPONSE_RATE', 'HOST_ACCEPTANCE_RATE',
       'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT', 'HOST_ENTIRE_HOMES',
       'HOST_PRIVATE_ROOMS', 'HOST_SHARED_ROOMS', 'HOST_VERIFICATIONS',
       'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED', 'STREET',
       'NEIGHBOURHOOD_CLEANSED', 'CITY', 'STATE', 'ZIPCODE', 'MARKET',
       'SMART_LOCATION', 'COUNTRY', 'LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT', 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES',
       'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'AMENITIES', 'SQUARE_FEET',
       'PRICE', 'WEEKLY_PRICE', 'MONTHLY_PRICE', 'SECURITY_DEPOSIT',
       'CLEANING_FEE', 'GUESTS_INCLUDED', 'EXTRA_PEOPLE', 'MINIMUM_NIGHTS',
       'MAXIMUM_NIGHTS', 'MINIMUM_MINIMUM_NIGHTS', 'MAXIMUM_MINIMUM_NIGHTS',
       'MINIMUM_MAXIMUM_NIGHTS', 'MAXIMUM_MAXIMUM_NIGHTS',
       'MINIMUM_NIGHTS_AVG_NTM', 'MAXIMUM_NIGHTS_AVG_NTM', 'CALENDAR_UP

In [ ]:
# upload FS_LIST_RAW

fs_upload(engine,df_list,'FS_LIST_RAW')

In [ ]:
# features for model

col_list = ['id', 'last_scraped', 'host_since','host_neighbourhood',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'calculated_host_listings_count',
         'host_verifications','host_has_profile_pic', 'host_identity_verified', 
         'street','neighbourhood_cleansed',
        'city', 'state', 'zipcode', 'market', 'country', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 
       'price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'calendar_updated',
       'number_of_reviews','number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value','instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 
        'reviews_per_month']

In [41]:
# calendar data

# read files
all_cal= glob.glob(os.path.join(path,"calendar*.csv"))

ls_cal = []
for file in all_cal:
    df = pd.read_csv(file,header=0)
    #df['SCRAPED_DATE'] = df.date.min()
    ls_cal.append(df)

    
# for each monthly scrape, only keep the calendar data b/w the scraped date and the date of next scrape run
ls_month = []
num_cal = len(ls_cal)
for i in range(num_cal):
    if i < num_cal-1:
        date_end = ls_cal[i+1].date.min()
        df = ls_cal[i]
        df = df[df.date < date_end]
    else:
        df = ls_cal[i]
        date_start = pd.to_datetime(df.date.min()).date()
        date_end = date_start+relativedelta(months=1)
        df = df[df.date<str(date_end)]
    ls_month.append(df)

df_cal = pd.concat(ls_month,axis=0,ignore_index=True)


# clean
df_cal.columns = df_cal.columns.str.upper()
df_cal['ADJUSTED_PRICE'] = df_cal['ADJUSTED_PRICE'].str.replace('$','')
df_cal['ADJUSTED_PRICE'] = df_cal['ADJUSTED_PRICE'].str.replace(',','')
df_cal['ADJUSTED_PRICE']=df_cal['ADJUSTED_PRICE'].astype(float) # convert to float to keep nan values
df_cal['PRICE'] = df_cal['PRICE'].str.replace('$','')
df_cal['PRICE'] = df_cal['PRICE'].str.replace(',','')
df_cal['PRICE']=df_cal['PRICE'].astype(float)
df_cal['BOOKED'] = df_cal.AVAILABLE.replace({'t':0,'f':1})

In [29]:
df_cal.head()

,LISTING_ID,DATE,AVAILABLE,PRICE,ADJUSTED_PRICE,MINIMUM_NIGHTS,MAXIMUM_NIGHTS,BOOKED
0,251965,2020-01-07,f,259.0,259.0,1.0,365.0,1
1,653384,2020-01-07,f,100.0,100.0,1.0,1125.0,1
2,653384,2020-01-08,f,100.0,100.0,1.0,1125.0,1
3,653384,2020-01-09,f,100.0,100.0,1.0,1125.0,1
4,653384,2020-01-10,f,100.0,100.0,1.0,1125.0,1


In [43]:
df_cal.shape

(4555609, 9)

In [28]:
# upload FS_CAL_RAW

engine = connect_my_db('secrets/db_string')
fs_upload(engine,df_cal,'FS_CAL_RAW')

1143.5180366039276

In [48]:
# aggregate to monthly

df_cal['YEAR_MONTH'] = df_cal.DATE.str[:7]

df_month = df_cal[['LISTING_ID','YEAR_MONTH','ADJUSTED_PRICE','BOOKED']].groupby(['LISTING_ID','YEAR_MONTH']).\
agg({'ADJUSTED_PRICE':'median','BOOKED':'sum'}).reset_index()

# drop the half last month
df_month=df_month[~(df_month.YEAR_MONTH=='2020-07')]

In [56]:
df_month

,LISTING_ID,YEAR_MONTH,ADJUSTED_PRICE,BOOKED
0,9835,2020-01,60.0,0
1,9835,2020-02,60.0,0
2,9835,2020-03,60.0,0
3,9835,2020-04,60.0,0
4,9835,2020-05,60.0,0
...,...,...,...,...
180369,43733191,2020-06,320.0,11
180371,43744200,2020-06,55.0,15
180373,43746933,2020-06,140.0,7
180375,43748923,2020-06,79.0,19


In [ ]:
# upload FS_CAL_MONTHLY

fs_upload(engine,df_cal,'FS_CAL_MONTHLY')

In [ ]:
fre=cal.listing_id.value_counts()

In [ ]:
cal.available=cal.available.replace({'t':1,'f':0})
cal.head()

In [ ]:
cal.date.max()

In [ ]:
cal[['listing_id','available']].groupby(['listing_id']).sum()

In [ ]:
fre.unique()

In [6]:
fre.unique()

In [9]:
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,323280,2020-06-12,t,$67.00,$67.00,1.0,730365.0
1,315513,2020-06-12,f,$45.00,$45.00,1.0,1125.0
2,315513,2020-06-13,f,$45.00,$45.00,1.0,1125.0
3,315513,2020-06-14,f,$45.00,$45.00,1.0,1125.0
4,315513,2020-06-15,f,$45.00,$45.00,1.0,1125.0


In [10]:
fre=cal.listing_id.value_counts()

In [19]:
cal.available=cal.available.replace({'t':1,'f':0})
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,323280,2020-06-12,1,$67.00,$67.00,1.0,730365.0
1,315513,2020-06-12,0,$45.00,$45.00,1.0,1125.0
2,315513,2020-06-13,0,$45.00,$45.00,1.0,1125.0
3,315513,2020-06-14,0,$45.00,$45.00,1.0,1125.0
4,315513,2020-06-15,0,$45.00,$45.00,1.0,1125.0


In [21]:
cal.date.max()

'2021-06-13'

In [20]:
cal[['listing_id','available']].groupby(['listing_id']).sum()

,available
listing_id,
9835,365
10803,41
12936,0
38271,351
41836,0
...,...
43733191,79
43744200,4
43746933,173


In [15]:
fre.unique()

array([367, 366, 365], dtype=int64)